In [ ]:
import os
import time
import qmt
import numpy as np

## Simple webapp demo to show how offline playback of data works

Except for the notebook-specific examples, this notebook shows the same examples as `webapp_example_script.py` but integrated in a Jupyter notebook.

Note that in contrast to the example script, `quiet=True` is set to disable the debug messages. If there are problems, set `quiet` to `False`!

The default `show` method does not work in notebooks. Instead, use `chromium` or `iframe`.

In [ ]:
# create some quaternion data for testing
t = qmt.timeVec(T=10, Ts=0.01)
axis = np.column_stack([np.cos(t), np.zeros_like(t), np.sin(t)])
quat = qmt.quatFromAngleAxis(np.sin(t), axis)
data = qmt.Struct(t=t, quat=quat)

# run webapp
webapp = qmt.Webapp('/view/imubox', data=data, show='chromium', quiet=True)
webapp.run()

## Jupyter Special: Show in iframe instead of window

In Jupyter notebooks, webapps can be shown in an iframe instead of a separate window.

**Important**: When using the iframe mode, in some cases the browser might show files from the cache instead of the current webapp. If in doubt, right click on the iframe and choose _This Frame > Reload Frame_.

In [ ]:
# create some quaternion data for testing
t = qmt.timeVec(T=10, Ts=0.01)
axis = np.column_stack([np.cos(t), np.zeros_like(t), np.sin(t)])
quat = qmt.quatFromAngleAxis(np.sin(t), axis)
data = qmt.Struct(t=t, quat=quat)

# run webapp
webapp = qmt.Webapp('/view/imubox', data=data, show='iframe', quiet=True)
webapp.iframeHeight=250
webapp.run()

## Jupyter Special: Update the data of the previous example

In [ ]:
# generate data with a way faster movement and send it to the webapp
t = qmt.timeVec(T=10, Ts=0.01)
axis = np.column_stack([np.cos(10*t), np.zeros_like(t), np.sin(10*t)])
quat = qmt.quatFromAngleAxis(np.sin(10*t), axis)
data = qmt.Struct(t=t, quat=quat)
webapp.data = data

## Example of how to use a config

By default, the /view/imubox webapp will detect all signals named "quat" or "quat" + one single letter, and
visualize them (with the single letter painted on the box). It is also possible to define a custom config that will
define which data is displayed.

In [ ]:
# create some quaternion data for testing
t = qmt.timeVec(T=10, Ts=0.01)
axis = np.column_stack([np.cos(t), np.zeros_like(t), np.sin(t)])
quat = qmt.quatFromAngleAxis(np.sin(t), axis)
quat2 = qmt.qinv(quat)
data = qmt.Struct(t=t, quat1=quat, quat2=quat2)

# This config tells the webapp how many IMU boxes to create and how the variables are called.
# Furthermore, we show the first quaternion twice and use a different IMU coordinate system for the second copy A'
# (x axis pointing forward instead of right). For the last IMU, we enable axis arrows.
# Markers can be specified to annotate specific parts of the generated data in the playback timeline.
config = {
    'imus': [
        {'signal': 'quat1', 'letter': 'A'},
        {'signal': 'quat1', 'letter': 'A\'', 'cs': 'FLU'},
        {'signal': 'quat2', 'letter': 'B', 'axes': True},
    ],
    'markers': [{'pos': 4, 'end': 6, 'name': 'example for a range marker'}],
}

webapp = qmt.Webapp('/view/imubox', config=config, data=data, show='iframe', quiet=True)
webapp.iframeHeight=250
webapp.run()

## Simple example for online data processing

We use a ClockDataSource to generate samples at a fixed interval and create a simple Block class that does the
    online data processing. The ClockDataSource generates samples that only contain the time, but we could also use
    data sources that provide real-time measurement data from IMUs (e.g. via bluetooth).

In [ ]:
class ExampleBlock(qmt.Block):
    def step(self, inputs):
        t = inputs['t']
        axis = [np.cos(t), np.zeros_like(t), np.sin(t)]
        quat = qmt.quatFromAngleAxis(np.sin(t), axis)
        return {'t': t, 'quat': quat.flatten()}

webapp = qmt.Webapp('/view/imubox', show='iframe', quiet=True)
webapp.setupOnlineLoop(qmt.ClockDataSource(0.04), ExampleBlock())
webapp.iframeHeight=180
webapp.run()

## Simple example for the dummy IMU data souce

The DummyImuDataSource generates artificial IMU data. In this example, we just show the quaternions generated by
the data source, but we could easily add a processing block (like in the example above) to perform custom data
processing based on the gyroscope, accelerometer, and magnetometer measurements.

In [ ]:
webapp = qmt.Webapp('/view/imubox', show='iframe', quiet=True)
webapp.setupOnlineLoop(qmt.DummyImuDataSource(0.04, 3))
# We could also use qmt.dataSourceFromJson to dynamically create the data source from a JSON configuration string.
# This can be used to allow for loading a data source that accesses real IMU data from external module, and define
# this module and the necessary parameters via a command line argument.
# webapp.setupOnlineLoop(qmt.dataSourceFromJson('{"class": "qmt.DummyImuDataSource", "Ts": 0.04, "N": 3}'))
webapp.iframeHeight=180
webapp.run()

## Example of how to play back existing data from Python

Unlike the previous offline examples, playback is handed on the Python side and only the current sample is
transferred via the websocket. This is for example useful to replay experiments in order to create
high quality videos.

Note that we can also combine this with online data processing, e.g. to replay a file containing sensor data and
processing it in real-time.

In [ ]:
# generate .mat file with data for playback
t = qmt.timeVec(T=10, Ts=0.01)
axis = np.column_stack([np.cos(t), np.zeros_like(t), np.sin(t)])
quat = qmt.quatFromAngleAxis(np.sin(t), axis)
quat2 = qmt.qinv(quat)
data = qmt.Struct(t=t, quat1=quat, quat2=quat2)
data.save('example_output/webapp_example_data.mat')

config = {
    'imus': [
        {'signal': 'quat1', 'letter': 'A'},
        {'signal': 'quat2', 'letter': 'B'},
    ]
}

# let's also save the config to a file. this way, you can test the command line tool:
# qmt-webapp /view/imubox -d example_output/webapp_example_data.mat -c example_output/webapp_example_config.mat
qmt.Struct(config).save('example_output/webapp_example_config.mat')

webapp = qmt.Webapp('/view/imubox', config=config, show='iframe', quiet=True)
# Note that saving to a file is not necessary and we can just directly pass the data object.
# We could also use qmt.PlaybackDataSource to control addtional parameters (e.g. looping).
webapp.setupOnlineLoop('example_output/webapp_example_data.mat')
webapp.iframeHeight=200
webapp.run()

## Example for interactive offline batch processing

With the show_speed_slider option, we tell the visualisation to create a slider. Every time the value is changed,
we recalculate the whole data and send the new data to the visualization.

In [ ]:
def processData(webapp, params):
    speed = params['speed']
    t = qmt.timeVec(T=10, Ts=0.01)
    axis = np.column_stack([np.cos(speed*t), np.zeros_like(t), np.sin(speed*t)])
    quat = qmt.quatFromAngleAxis(np.sin(speed*t), axis)
    webapp.data = qmt.Struct(t=t, quat=quat)

config = {'show_speed_slider': True}
webapp = qmt.Webapp('/view/imubox', config=config, show='iframe', quiet=True)
webapp.on('params', processData)
webapp.iframeHeight=280
webapp.run()

## Example for interactive online processing

With the show_speed_slider option, we tell the visualisation to create a slider. The setParam function of the
processing block will automatically be called every time the slider value changes.

In [ ]:
class ExampleBlock(qmt.Block):
    def __init__(self):
        super().__init__()
        self.params['speed'] = 1.0

    def step(self, sample):
        t = sample['t']
        speed = self.params['speed']
        axis = [np.cos(speed*t), 0, np.sin(speed*t)]
        quat = qmt.quatFromAngleAxis(np.sin(speed * t), axis)
        return {'t': t, 'quat': quat}

config = {'show_speed_slider': True}
webapp = qmt.Webapp('/view/imubox', config=config, show='iframe', quiet=True)
webapp.setupOnlineLoop(qmt.ClockDataSource(0.04), ExampleBlock())
webapp.iframeHeight=230
webapp.run()

## Example for interactive online processing in a separate process

The function run is executed in a separate process via the ProcessDataSource and the multiprocessing package. The run function can use a qmt.ProcessDataSourceConnection object to communicate with the webapp. This makes it possible to integrate existing code that is written in a blocking way and does not function well in combination with async code.

In [ ]:
def run(conn, Ts):
    t0 = time.monotonic()
    N = 0
    while True:
        t = round(N*Ts, 9)
        time.sleep(max(0, t0 + t - time.monotonic()))
        N += 1

        # receive parameters (and commands) from webapp
        try:
            speed = conn.getParams().get('speed', 1.0)
            for command in conn.getCommands():
                print('received command:', command)
        except KeyboardInterrupt:
            return  # stop process gracefully

        # calculate and send sample
        axis = [np.cos(speed * t), 0, np.sin(speed * t)]
        quat = qmt.quatFromAngleAxis(np.sin(speed * t), axis)
        conn.sendSample({'t': t, 'quat': quat})

config = {'show_speed_slider': True, 'imus': [{'signal': 'quat'}]}
webapp = qmt.Webapp('/view/imubox', config=config, show='iframe', quiet=True)
webapp.setupOnlineLoop(qmt.ProcessDataSource(run, 0.04))
webapp.iframeHeight=230
webapp.run()